In [1]:
import pandas as pd
import numpy as np

oed_scalar_columns = ['id','scenario_id','region','input_energy_vector','output_energy_vector','parameter_name','technology',
                      'technology_type','value','unit','tags','method','source','comment','year']
oed_scalar_output = pd.DataFrame(columns=oed_scalar_columns)


id_scalar = 1
years = [2016,2030,2050] # make a list of the possible years
IDs = ['XX'] #enter scenario id
source ='urbs'
base_vari_dict = {2016: '1', 2030: '2', 2050: '9'} # dictionary for scenario variation ID's
try:
    vari = IDs[0][-1]
    vari = str(int(vari)+2)
except:
    vari = base_vari_dict[2050]
    
    
oed_timeseries_columns = ['id','scenario_id','region','input_energy_vector','output_energy_vector','parameter_name','technology',
                      'technology_type','timeindex_start','timeindex_stop','timeindex_resolution','series','unit','tags','method','source','comment']
oed_timeseries_output = pd.DataFrame(columns=oed_timeseries_columns)
id_timeseries = 1

In [2]:
year = years[1]
ID = IDs[0]
scenario_id = IDs[0][0:2]

In [3]:
urbs_orig = pd.read_excel("Urbs outputs/urbs_output_"+str(year)+"_ID"+ID+".xlsx", sheet_name=None)

In [4]:
input_orig = pd.read_excel("Urbs inputs/"+str(year)+"_ID_"+ID+".xlsx", sheet_name=None)

In [5]:
urbs={}

In [6]:
process_commodity = input_orig['Process-Commodity']
transmission = input_orig['Transmission']
sites = input_orig['Site']
costs = urbs_orig['Costs']

In [7]:
transmission.set_index(['Site In','Site Out','Transmission','Commodity'],inplace=True)

In [8]:
created = {}
storage = {}
imports ={}
exports = {}

In [9]:
for sheet in urbs_orig:
    urbs[sheet] = urbs_orig[sheet].fillna(method='ffill', axis=0)

In [10]:
commodity_sums = urbs['Commodity sums']
commodity_sums.set_index(['Unnamed: 0','Unnamed: 1'],inplace=True)

In [11]:
process_caps = urbs['Process caps']
process_caps.set_index(['Stf','Site','Process'],inplace=True)

storage_caps = urbs['Storage caps']
storage_caps.set_index(['Stf','Site','Storage','Commodity'],inplace=True)

trans_caps = urbs['Transmission caps']
trans_caps.set_index(['Stf','Site In','Site Out','Transmission','Commodity'],inplace=True)


In [13]:
#parse timeseries values
for sheet in urbs:
    if sheet.startswith('2'):
        region_name=sheet.split('.')[1]
        urbs[sheet].columns = urbs[sheet].columns.to_series().replace('Unnamed:\s\d+',np.nan,regex=True).ffill().values
        created[region_name]=urbs[sheet]['Created'][1:]
        created[region_name].columns= created[region_name].iloc[0]
        created[region_name] = created[region_name][1:]
        created[region_name].drop(columns='Stock', inplace=True)
        sheet_wo_timeseries = sheet.split(' ', 1)[0]
        
        try:
            storage[region_name] = urbs[sheet]['Storage'][1:]
            storage[region_name].columns = storage[region_name].iloc[0]
            storage[region_name] = storage[region_name][1:]
        except:
            pass
            
        try:
            imports[region_name] = urbs[sheet]['Import from'][1:]
            imports[region_name].columns = imports[region_name].iloc[0]
            imports[region_name] = imports[region_name][1:]
        except:
            pass
        
        try:
            exports[region_name] = urbs[sheet]['Export to'][1:]
            exports[region_name].columns = exports[region_name].iloc[0]
            exports[region_name] = exports[region_name][1:]
        except:
            pass
 

In [14]:
for sheet in urbs:
    if sheet.startswith('2'):
        region_name=sheet.split('.')[1]
        sheet_wo_timeseries = sheet.split(' ', 1)[0]
        
        non_slack_techs = (x for x in process_caps[np.in1d(process_caps.index.get_level_values(1), [region_name])].index.get_level_values(2) if (x != 'Slack' and x!='Curtailment'))

        for tech in non_slack_techs:
            #process capacities (new)
            oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': tech.split('_')[0],
                                      'output_energy_vector':'electricity','parameter_name':'added capacity',
                                      'technology': tech.split('_')[1],
                                      'technology_type': tech.split('_')[2],
                                      'value':process_caps.loc[year, region_name, tech]['New'] / 1000,
                                      'unit': 'GW',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year},ignore_index=True)
            id_scalar += 1
#
            #process capacities (total)
            oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': tech.split('_')[0],
                                      'output_energy_vector':'electricity','parameter_name':'capacity',
                                      'technology': tech.split('_')[1],
                                      'technology_type': tech.split('_')[2],
                                      'value': process_caps.loc[year, region_name, tech]['Total'] / 1000,
                                      'unit': 'GW',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year},ignore_index=True)
            id_scalar += 1
            
        non_slack_techs = (x for x in created[region_name].columns if x != 'Slack')

        for tech in non_slack_techs:    
            #generations
            oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': tech.split('_')[0],
                                      'output_energy_vector':'electricity','parameter_name':'electricity generation',
                                      'technology': tech.split('_')[1],
                                      'technology_type': tech.split('_')[2],
                                      'value':created[region_name][tech].sum()/1000,
                                      'unit': 'GWh',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year},ignore_index=True)
            id_scalar += 1
            
            #emissions
            if tech in list(process_commodity.loc[(process_commodity['Commodity'] == 'co2')]['Process']):
                tech_emission = float(process_commodity.loc[(process_commodity['Commodity'] == 'co2') & (process_commodity['Process'] == tech)]['ratio'])
                tech_elec = float(process_commodity.loc[(process_commodity['Commodity'] == 'electricity') & (process_commodity['Process'] == tech)]['ratio'])                
                oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': tech.split('_')[0],
                                      'output_energy_vector':'electricity',
                                      'parameter_name':'emissions',
                                      'technology': tech.split('_')[1],
                                      'technology_type': tech.split('_')[2],
                                      'value':created[region_name][tech].sum()/tech_elec*tech_emission/10**9,
                                      'unit': 'Gt/a',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year},ignore_index=True)   
                id_scalar += 1   
                
            #primary energy   
            prim_energy = float(sum(process_commodity.loc[(process_commodity['Process'] == tech) 
                                                          & (process_commodity['Direction'] == 'In')]['ratio']))
            oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': tech.split('_')[0],
                                      'output_energy_vector':'electricity',
                                      'parameter_name':'primary energy consumption',
                                      'technology': tech.split('_')[1],
                                      'technology_type': tech.split('_')[2],
                                      'value':created[region_name][tech].sum()*prim_energy,
                                      'unit': 'MWh/a',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year},ignore_index=True)    
        
            id_scalar += 1  
        
        
   
        #storage input energy
        oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity',
                                      'parameter_name':'input energy',
                                      'technology': 'storage',
                                      'technology_type': 'all',
                                      'value':  commodity_sums.loc['Storage','Stored'][sheet_wo_timeseries]/10**3,
                                      'unit': 'GWh',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year},ignore_index=True)  
        id_scalar += 1   
        
        #storage output energy
        oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity',
                                      'parameter_name':'output energy',
                                      'technology': 'storage',
                                      'technology_type': 'all',
                                      'value':  commodity_sums.loc['Storage','Retrieved'][sheet_wo_timeseries]/10**3,
                                      'unit': 'GWh',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year},ignore_index=True)  
        id_scalar += 1 
        
        #storage losses
        oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity',
                                      'parameter_name':'losses',
                                      'technology': 'storage',
                                      'technology_type': 'all',
                                      'value':  commodity_sums.loc['Storage','Stored'][sheet_wo_timeseries]/10**3
                                                      - commodity_sums.loc['Storage','Retrieved'][sheet_wo_timeseries]/10**3,
                                      'unit': 'GWh',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year},ignore_index=True)  
        id_scalar += 1           

        #energy flow
        expor = commodity_sums[sheet_wo_timeseries].loc['Export'][commodity_sums[sheet_wo_timeseries].loc['Export']!=0]
        impor = commodity_sums[sheet_wo_timeseries].loc['Import'][commodity_sums[sheet_wo_timeseries].loc['Import']!=0]

        net_export = expor.sub(impor, fill_value=0)
        for neighbor in net_export.index:
            if net_export[neighbor] > 0:
                oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [region_name,neighbor],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity',
                                      'parameter_name':'energy flow',
                                      'technology': 'transmission',
                                      'technology_type': 'HVAC',
                                      'value':  net_export[neighbor]/10**3,
                                      'unit': 'GWh',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)     
                id_scalar += 1 
            
        #trans losses
        for neighbor in expor.index:  
            try:
                oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [region_name,neighbor],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity',
                                      'parameter_name':'losses',
                                      'technology': 'transmission',
                                      'technology_type': 'HVAC',
                                      'value':  expor[neighbor] * 
                                            (1 - transmission.loc[(region_name,neighbor,'hvac','electricity')]['eff']),
                                      'unit': 'MWh/a',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)     
            except:    
                oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [region_name,neighbor],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity',
                                      'parameter_name':'losses',
                                      'technology': 'transmission',
                                      'technology_type': 'HVAC',
                                      'value':  expor[neighbor] * 
                                            (1 - transmission.loc[(region_name,neighbor,'DC','electricity')]['eff']),
                                      'unit': 'MWh/a',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)            
            id_scalar += 1 
            
#emissions total
oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': list(sites['Name']),'input_energy_vector': 'ALL',
                                      'output_energy_vector':'co2',
                                      'parameter_name':'emissions',
                                      'technology': 'ALL',
                                      'technology_type': 'ALL',
                                      'value':  sum(oed_scalar_output[oed_scalar_output['parameter_name'] == 'emissions']['value']),
                                      'unit': 'Gt/a',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)             
id_scalar += 1 

#costs total   
oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': list(sites['Name']),'input_energy_vector': 'ALL',
                                      'output_energy_vector':'ALL',
                                      'parameter_name':'system cost',
                                      'technology': 'ALL',
                                      'technology_type': 'ALL',
                                      'value':  sum(costs['costs']),
                                      'unit': '€/a',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True) 
id_scalar += 1

oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': list(sites['Name']),'input_energy_vector': 'ALL',
                                      'output_energy_vector':'ALL',
                                      'parameter_name':'investment cost',
                                      'technology': 'ALL',
                                      'technology_type': 'ALL',
                                      'value':  costs[costs.cost_type=='Invest'].iloc[0][1],
                                      'unit': '€/a',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True) 
id_scalar += 1

oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': list(sites['Name']),'input_energy_vector': 'ALL',
                                      'output_energy_vector':'ALL',
                                      'parameter_name':'fixed cost',
                                      'technology': 'ALL',
                                      'technology_type': 'ALL',
                                      'value':  costs[costs.cost_type=='Fixed'].iloc[0][1],
                                      'unit': '€/a',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True) 
id_scalar += 1

oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': list(sites['Name']),'input_energy_vector': 'ALL',
                                      'output_energy_vector':'ALL',
                                      'parameter_name':'variable cost',
                                      'technology': 'ALL',
                                      'technology_type': 'ALL',
                                      'value':  (costs[costs.cost_type=='Variable'].iloc[0][1] + 
                                              costs[costs.cost_type=='Fuel'].iloc[0][1] + 
                                                 costs[costs.cost_type=='Environmental'].iloc[0][1]),
                                      'unit': '€/a',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True) 
id_scalar += 1

#curtailment
try:
    oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': list(sites['Name']),'input_energy_vector': 'ALL',
                                      'output_energy_vector':'ALL',
                                      'parameter_name':'curtailment',
                                      'technology': 'ALL',
                                      'technology_type': 'ALL',
                                      'value':  sum(commodity_sums.loc['Consumed','Curtailment']) / 1000,
                                       'unit': 'GWh',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)
except: #no curtailment done
    oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': list(sites['Name']),'input_energy_vector': 'ALL',
                                      'output_energy_vector':'ALL',
                                      'parameter_name':'curtailment',
                                      'technology': 'ALL',
                                      'technology_type': 'ALL',
                                      'value':  0,
                                      'unit': 'GWh',                                                  
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)
id_scalar += 1 
    
#slack
try:
    oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': list(sites['Name']),'input_energy_vector': 'ALL',
                                      'output_energy_vector':'ALL',
                                      'parameter_name':'slack',
                                      'technology': 'ALL',
                                      'technology_type': 'ALL',
                                      'value':  sum(commodity_sums.loc['Created','Slack'])/10**3,
                                      'unit': 'GWh',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)
except: #no slack generated
    oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': list(sites['Name']),'input_energy_vector': 'ALL',
                                      'output_energy_vector':'ALL',
                                      'parameter_name':'slack',
                                      'technology': 'ALL',
                                      'technology_type': 'ALL',
                                      'value':  0,
                                      'unit': 'GWh',                                                  
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)
id_scalar += 1 

for (stf,sit,sto,com) in storage_caps.index:
    #storage capacities (new, c)
    oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [sit],'input_energy_vector': com,
                                      'output_energy_vector': com, 'parameter_name':'added capacity',
                                      'technology': 'storage',
                                      'technology_type': sto,
                                      'value': storage_caps.loc[stf, sit, sto, com]['C New'] /10**3,
                                      'unit': 'GWh',
                                      'method': '{"value": "scalar"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)
    id_scalar += 1 
        
    #storage capacities (new, p)
    oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [sit],'input_energy_vector': com,
                                      'output_energy_vector': com, 'parameter_name':'added power capacity',
                                      'technology': 'storage',
                                      'technology_type': sto,
                                      'value': storage_caps.loc[stf, sit, sto, com]['P New'] /10**3,
                                      'unit': 'GW',
                                      'method': '{"value": "scalar"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)
    id_scalar += 1      
 

    #storage capacities (total, c)
    oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [sit],'input_energy_vector': com,
                                      'output_energy_vector': com, 'parameter_name':'capacity',
                                      'technology': 'storage',
                                      'technology_type': sto,
                                      'value': storage_caps.loc[stf, sit, sto, com]['C Total'] /10**3,
                                      'unit': 'GWh',
                                      'method': '{"value": "scalar"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)
    id_scalar += 1 
        
    #storage capacities (total, p)
    oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [sit],'input_energy_vector': com,
                                      'output_energy_vector': com, 'parameter_name':'power capacity',
                                      'technology': 'storage',
                                      'technology_type': sto,
                                      'value': storage_caps.loc[stf, sit, sto, com]['P Total'] /10**3,
                                      'unit': 'GW',
                                      'method': '{"value": "scalar"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)
    id_scalar += 1 
        
for (stf,sitin,sitout,tra,com) in trans_caps.index:
    #transmission capacities (new)
    oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [sitin,sitout],'input_energy_vector': com,
                                      'output_energy_vector': com, 'parameter_name':'added capacity',
                                      'technology': 'transmission',
                                      'technology_type': tra,
                                      'value': trans_caps.loc[stf, sitin, sitout, tra, com]['New'] /10**3,
                                      'unit': 'GW',
                                      'method': '{"value": "scalar"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)
    id_scalar += 1 
        
    #transmission capacities (total)
    oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': [sitin,sitout],'input_energy_vector': com,
                                      'output_energy_vector': com, 'parameter_name':'capacity',
                                      'technology': 'transmission',
                                      'technology_type': tra,
                                      'value': trans_caps.loc[stf, sitin, sitout, tra, com]['Total'] /10**3,
                                      'unit': 'GW',
                                      'method': '{"value": "scalar"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True)
    id_scalar += 1      
 
#renewable generation
re_list = ['wind turbine','hydro turbine','photovoltaics']
ren_gen = oed_scalar_output[(oed_scalar_output.parameter_name == 'electricity generation')]
ren_gen = ren_gen[ren_gen.technology.isin(re_list)]

#ren_gen['value'].sum()
oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
                                      'region': list(sites['Name']),'input_energy_vector': 'ALL',
                                      'output_energy_vector':'ALL',
                                      'parameter_name':'renewable generation',
                                      'technology': 'ALL',
                                      'technology_type': 'ALL',
                                      'value':  ren_gen['value'].sum(),
                                      'unit': 'GWh',
                                      'method': '{"value": "aggregated"}',
                                      'source': 'urbs',
                                      'year': year}, ignore_index=True) 
id_scalar += 1


In [17]:
# timeseries for year 2016

from datetime import datetime
t_start=datetime(year, 1, 1)
t_stop=datetime(year, 12, 31)

timeindex_start = t_start.strftime("%Y-%m-%d %H:%M:%S+00:00")
timeindex_stop = t_stop.strftime("%Y-%m-%d %H:%M:%S+00:00")

for sheet in urbs:
    if sheet.startswith('2'):
        region_name=sheet.split('.')[1]
        sheet_wo_timeseries = sheet.split(' ', 1)[0]

        non_slack_techs = (x for x in created[region_name].columns if x != 'Slack')
        
        #generations
        for tech in non_slack_techs:
            oed_timeseries_output = oed_timeseries_output.append({'id': id_timeseries, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': tech.split('_')[0],
                                      'output_energy_vector':'electricity', 'parameter_name':'electricity generation',
                                      'technology': tech.split('_')[1],
                                      'technology_type': tech.split('_')[2],
                                      'timeindex_start':timeindex_start,
                                      'timeindex_stop':timeindex_stop,
                                      'timeindex_resolution': '1h',
                                      'series': list(created[region_name][tech]),                                             
                                      'unit': 'MWh/h',
                                      'method': '{"value": "timeseries"}',
                                      'source': 'urbs'} ,ignore_index=True)
            id_timeseries += 1
            
            #emissions
          # if tech in list(process_commodity.loc[(process_commodity['Commodity'] == 'co2')]['Process']):
           #     tech_emission = float(process_commodity.loc[(process_commodity['Commodity'] == 'co2') & (process_commodity['Process'] == tech)]['ratio'])
          #      oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
          #                            'region': region_name,'input_energy_vector': tech.split('_')[0],
           #                           'output_energy_vector':'electricity',
           #                           'parameter_name':'emissions',
           #                           'technology': tech.split('_')[1],
           #                           'technology_type': tech.split('_')[2],
           #                           'value':created[region_name][tech].sum()*tech_emission/10**9,
           #                           'unit': 'Gt/a',
           #                           'method': '{"value": "aggregated"}',
           #                           'source': 'urbs',
           #                           'year': year},ignore_index=True)   
           #     id_scalar += 1   
                
            #primary energy   
            #prim_energy = float(process_commodity.loc[(process_commodity['Process'] == tech) & (process_commodity['Direction'] == 'In')]['ratio'])
            #oed_scalar_output = oed_scalar_output.append({'id': id_scalar, 'scenario_id': scenario_id,
           #                           'region': region_name,'input_energy_vector': tech.split('_')[0],
            #                          'output_energy_vector':'electricity',
            #                          'parameter_name':'primary energy consumption',
            #                          'technology': tech.split('_')[1],
            #                          'technology_type': tech.split('_')[2],
            #                          'value':created[region_name][tech].sum()*prim_energy,
            #                          'unit': 't/a',
            #                          'method': '{"value": "aggregated"}',
             #                         'source': 'urbs',
             #                         'year': year},ignore_index=True)    
        
           # id_scalar += 1  
            
        #storage input energy
        if storage[region_name]['Stored'].iloc[0] == 'Stored': 
            oed_timeseries_output = oed_timeseries_output.append({'id': id_timeseries, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity', 'parameter_name':'input energy',
                                      'technology': 'storage',
                                      'technology_type': 'all',
                                      'timeindex_start': timeindex_start,
                                      'timeindex_stop': timeindex_stop,
                                      'timeindex_resolution': '1h',
                                      'series': [0] * len(storage[region_name]['Stored']),                                             
                                      'unit': 'MWh/h',
                                      'method': '{"value": "timeseries"}',
                                      'source': 'urbs'} ,ignore_index=True)              
        else:
            oed_timeseries_output = oed_timeseries_output.append({'id': id_timeseries, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity', 'parameter_name':'input energy',
                                      'technology': 'storage',
                                      'technology_type': 'all',
                                      'timeindex_start': timeindex_start,
                                      'timeindex_stop': timeindex_stop,
                                      'timeindex_resolution': '1h',
                                      'series': list(storage[region_name]['Stored']),                                             
                                      'unit': 'MWh/h',
                                      'method': '{"value": "timeseries"}',
                                      'source': 'urbs'} ,ignore_index=True)                     
        id_timeseries += 1
        
        #storage output energy
        if storage[region_name]['Retrieved'].iloc[0] == 'Retrieved': 
            oed_timeseries_output = oed_timeseries_output.append({'id': id_timeseries, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity', 'parameter_name':'output energy',
                                      'technology': 'storage',
                                      'technology_type': 'all',
                                      'timeindex_start': timeindex_start,
                                      'timeindex_stop': timeindex_stop,
                                      'timeindex_resolution': '1h',
                                      'series': [0] * len(storage[region_name]['Retrieved']),                                             
                                      'unit': 'MWh/h',
                                      'method': '{"value": "timeseries"}',
                                      'source': 'urbs'} ,ignore_index=True)
        else:    
            oed_timeseries_output = oed_timeseries_output.append({'id': id_timeseries, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity', 'parameter_name':'output energy',
                                      'technology': 'storage',
                                      'technology_type': 'all',
                                      'timeindex_start': timeindex_start,
                                      'timeindex_stop': timeindex_stop,
                                      'timeindex_resolution': '1h',
                                      'series': list(storage[region_name]['Retrieved']),                                             
                                      'unit': 'MWh/h',
                                      'method': '{"value": "timeseries"}',
                                      'source': 'urbs'} ,ignore_index=True)             
        id_timeseries += 1 

        #storage level
        if storage[region_name]['Level'].iloc[0] == 'Level': 
            oed_timeseries_output = oed_timeseries_output.append({'id': id_timeseries, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity', 'parameter_name':'storage level',
                                      'technology': 'storage',
                                      'technology_type': 'all',
                                      'timeindex_start': timeindex_start,
                                      'timeindex_stop': timeindex_stop,
                                      'timeindex_resolution': '1h',
                                      'series': [0] * len(storage[region_name]['Level']),                                             
                                      'unit': 'MWh/h',
                                      'method': '{"value": "timeseries"}',
                                      'source': 'urbs'} ,ignore_index=True)
        else:    
            oed_timeseries_output = oed_timeseries_output.append({'id': id_timeseries, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity', 'parameter_name':'storage level',
                                      'technology': 'storage',
                                      'technology_type': 'all',
                                      'timeindex_start': timeindex_start,
                                      'timeindex_stop': timeindex_stop,
                                      'timeindex_resolution': '1h',
                                      'series': list(storage[region_name]['Level']),                                             
                                      'unit': 'MWh/h',
                                      'method': '{"value": "timeseries"}',
                                      'source': 'urbs'} ,ignore_index=True)             
        id_timeseries += 1         
        
        #energy flows         
        if 'Export to' in urbs[sheet].columns:
            try:
                looper = exports[region_name].columns
            except:
                looper = [exports[region_name].name]
        for neighbor in looper:
            try:
                oed_timeseries_output = oed_timeseries_output.append({'id': id_timeseries, 'scenario_id': scenario_id,
                                      'region': [region_name],'input_energy_vector': 'electricity',
                                      'output_energy_vector':'electricity', 'parameter_name':'power flow',
                                      'technology': 'transmission',
                                      'technology_type': 'hvac',
                                      'timeindex_start': timeindex_start,
                                      'timeindex_stop': timeindex_stop,
                                      'timeindex_resolution': '1h',
                                      'series': list(exports[region_name][neighbor]),                                             
                                      'unit': 'MWh/h',
                                      'method': '{"value": "timeseries"}',
                                      'source': 'urbs'} ,ignore_index=True) 
            except:
                pass
            id_timeseries += 1 


In [18]:
oed_scalar_output.tags = '{"scenario": "ID'+str(vari)+'"}'
oed_timeseries_output.tags = '{"scenario": "ID'+str(vari)+'"}'


In [19]:
oed_timeseries_output.to_csv('OED outputs/'+str(ID)+'/'+str(year)+'/oed_timeseries_output.csv',index=False, sep=';', quotechar = '"')

In [20]:
oed_scalar_output.to_csv('OED outputs/'+str(ID)+'/'+str(year)+'/oed_scalar_output.csv',index=False, sep=';', quotechar = '"')